Piotr Rzeźnik 402194

### Imports

In [171]:
import itertools
from tqdm.notebook import tqdm
import json
from datasets import load_dataset
from elasticsearch import Elasticsearch


### Load dataset

In [172]:
dataset = load_dataset("clarin-knext/fiqa-pl", 'corpus')
pd_dataset = dataset['corpus'].to_pandas()

### Configure ElasticSearch

In [173]:


ELASTIC_PASSWORD = 'tZSE8E-JJ+*FT07ID30Q'
client = Elasticsearch(
    "https://localhost:9200",
    ca_certs="../../http_ca.crt",
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

client.info()


ObjectApiResponse({'name': '8aa7680e5b7b', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'GUnOVmh2TI6YXU9DaPLQ4g', 'version': {'number': '8.8.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '98e1271edf932a480e4262a471281f1ee295ce6b', 'build_date': '2023-06-26T05:16:16.196344851Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Define two ES analyzers for Polish according to requirements.
### Define an ES index for storing the contents of the corpus from lab 1 using both analyzers. 

Analyzers and other settings are defined in `settings.json` file. 
Synonyms used in analyser are defined in `synonyms.txt` file 


settings.json
```
{
  "analysis": {
    "analyzer": {
      "analyzer_with_synonym": {
        "type": "custom",
        "tokenizer": "standard",
        "filter": [
          "synonyms_filter",
          "lowercase",
          "morfologik_stem",
          "lowercase"
        ]
      },
      "analyzer_without_synonym": {
        "type": "custom",
        "tokenizer": "standard",
        "filter": [
          "lowercase",
          "morfologik_stem",
          "lowercase"
        ]
      },
      "analyzer_without_lematization": {
        "type": "custom",
        "tokenizer": "standard",
        "filter": [
          "synonyms_filter",
          "lowercase"
        ]
      },
      "analyzer_without_synonym_and_lematization": {
        "type": "custom",
        "tokenizer": "standard",
        "filter": [
          "lowercase"
        ]
      }
    },
    "filter": {
      "synonyms_filter": {
        "type": "synonym",
        "synonyms_path": "./synonyms.txt"
      }
    }
  }
}```

synonyms.txt
```I, styczeń, sty
II ,luty, lut
III, marzec, mar
IV, kwiecień, kwi
V, maj
VI, czerwiec, cze
VII, lipiec, lip
VIII, sierpień, sie
IX, wrzesień, wrz
X, październik, paź
XI, listopad, lis
XII, grudzień, gru```

In [175]:
! docker cp ./synonyms.txt es01:/usr/share/elasticsearch/config 

In [176]:
INDEX_NAME = "months_index"
settings = json.load(open('./settings.json', 'r'))
client.indices.delete(index=INDEX_NAME)
client.indices.create(index=INDEX_NAME, settings=settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'months_index'})

### Load the data to the ES index.

In [177]:
body = list(itertools.chain(*pd_dataset.apply(lambda x: ({'index': {'_id': x._id}}, {'title': x.title, 'text': x.text}) , axis=1)))
chunk_size = 200
for chunk in tqdm([body[i:i + chunk_size] for i in range(0, len(body), chunk_size)]):
    client.bulk(
        index=INDEX_NAME,
        body=chunk
    )

  0%|          | 0/577 [00:00<?, ?it/s]

### Determine the number of documents containing the word styczeń (in any form) including and excluding the synonyms.

In [189]:
c1 = client.count(index=INDEX_NAME,  analyzer='analyzer_with_synonym', q='text:styczeń')['count']
c2 = client.count(index=INDEX_NAME,  analyzer='analyzer_without_synonym', q='text:styczeń')['count']

In [190]:
print(f"""
    Styczeń matches with synonyms:       {c1}
    Styczeń matches withotu synonyms:    {c2}""")


    Styczeń matches with synonyms:       44123
    Styczeń matches withotu synonyms:    29


Synonyms for Styczeń contains also "I" which is "and" word in Polish, so count of matches with synonyms is inadequate.

In [191]:
c1 = client.count(index=INDEX_NAME,  analyzer='analyzer_with_synonym', q='text:sierpień')['count']
c2 = client.count(index=INDEX_NAME,  analyzer='analyzer_without_synonym', q='text:sierpień')['count']

In [192]:
print(f"""
    Sierpień matches with synonyms:       {c1}
    Sierpień matches withotu synonyms:    {c2}""")


    Sierpień matches with synonyms:       50
    Sierpień matches withotu synonyms:    22


Number of Sierpień occurences are more accurate and realistic.  

### Download the QA pairs for the FIQA dataset. 


In [209]:
QA_dataset = load_dataset("clarin-knext/fiqa-pl-qrels", 'corpus')


In [210]:
QA_dataset['train']

Dataset({
    features: ['query-id', 'corpus-id', 'score'],
    num_rows: 14166
})

I didn't manage to understand and join that data together, so I couldn't process analysis. 

**Questions:**

1. What are the strengths and weaknesses of regular expressions versus full text search regarding processing of text?

`Regular expressions are simpler, easier to configure and for easy patterns are fast, but it is hard to create expression that covers every form. ElasticSearch needs to be configured, indexes need to be defined and data is required to be stored in ES, but it supports tokenization, lematization, and has many build-in filters. Searching and counting is very fast thanks to database system and indexes. `

2. Is full text search applicable to the question answering problem? Show at least 3 examples from the corpus to support your claim.

` Connected to QA pairs - couldn't be answered without doing that part` 